In [9]:
import numpy as np
from typing import List, Tuple

class VEGA:
    """
    Vector Evaluated Genetic Algorithm (VEGA) implementation.
    This class implements VEGA for solving multi-objective optimization problems.
    """

    def __init__(self, pop_size: int, num_objectives: int, gene_size: int, 
                 crossover_rate: float, mutation_rate: float):
        """
        Initialize the VEGA.

        Args:
            pop_size (int): Size of the population. Should be divisible by num_objectives.
            num_objectives (int): Number of objectives to optimize.
            gene_size (int): Number of genes in each individual.
            crossover_rate (float): Probability of crossover occurring.
            mutation_rate (float): Probability of mutation for each gene.
        """
        assert pop_size % num_objectives == 0, "Population size should be divisible by number of objectives"
        self.pop_size = pop_size
        self.num_objectives = num_objectives
        self.subpop_size = pop_size // num_objectives
        self.gene_size = gene_size
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.population = self.initialize_population()

    def initialize_population(self) -> np.ndarray:
        """
        Initialize a random population.

        Returns:
            np.ndarray: A 2D array where each row represents an individual.
        """
        return np.random.rand(self.pop_size, self.gene_size)

    def evaluate_fitness(self, individual: np.ndarray) -> List[float]:
        """
        Evaluate the fitness of an individual.
        This is a placeholder and should be replaced with actual objective functions.

        Args:
            individual (np.ndarray): The individual to evaluate.

        Returns:
            List[float]: A list of fitness values, one for each objective.
        """
        return [np.sum(individual) for _ in range(self.num_objectives)]

    def vega_selection(self, fitness_values: np.ndarray) -> np.ndarray:
        """
        Perform VEGA selection by creating sub-populations based on each objective.

        Args:
            fitness_values (np.ndarray): Fitness values for the entire population.

        Returns:
            np.ndarray: Combined selected individuals from all sub-populations.
        """
        selected_indices = []

        for obj in range(self.num_objectives):
            # Sort based on the current objective
            obj_indices = np.argsort(fitness_values[:, obj])[:self.subpop_size]
            selected_indices.extend(obj_indices)

        return self.population[selected_indices]

    def crossover(self, parent1: np.ndarray, parent2: np.ndarray) -> np.ndarray:
        """
        Perform crossover between two parents.

        Args:
            parent1 (np.ndarray): First parent.
            parent2 (np.ndarray): Second parent.

        Returns:
            np.ndarray: Child produced by crossover.
        """
        if np.random.rand() < self.crossover_rate:
            crossover_point = np.random.randint(1, self.gene_size)
            child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        else:
            child = parent1.copy()
        return child

    def mutate(self, individual: np.ndarray) -> np.ndarray:
        """
        Perform mutation on an individual.

        Args:
            individual (np.ndarray): The individual to mutate.

        Returns:
            np.ndarray: The mutated individual.
        """
        mutation_mask = np.random.rand(self.gene_size) < self.mutation_rate
        individual[mutation_mask] = np.random.rand(np.sum(mutation_mask))
        return individual

    def run(self, generations: int) -> np.ndarray:
        """
        Run the VEGA for a specified number of generations.

        Args:
            generations (int): Number of generations to run.

        Returns:
            np.ndarray: The final population.
        """
        for _ in range(generations):
            fitness_values = np.array([self.evaluate_fitness(ind) for ind in self.population])
            
            # Perform VEGA selection
            selected_population = self.vega_selection(fitness_values)
            
            # Create new population through crossover and mutation
            new_population = []
            while len(new_population) < self.pop_size:
                parent_indices = np.random.choice(len(selected_population), 2, replace=False)
                parent1, parent2 = selected_population[parent_indices]
                child = self.crossover(parent1, parent2)
                child = self.mutate(child)
                new_population.append(child)
            
            self.population = np.array(new_population)
        
        return self.population

# Usage example
vega = VEGA(pop_size=100, num_objectives=2, gene_size=10, crossover_rate=0.8, mutation_rate=0.1)
final_population = vega.run(generations=50)

In [10]:
print(vega.initialize_population()[0])

[0.29711776 0.06874323 0.09317056 0.58317227 0.77200254 0.75330566
 0.59441791 0.12298709 0.41417076 0.52118714]
